In [1]:
%time
import scraper
import pandas as pd
s = scraper.Scraper()
s.path = '/Users/sreetam'
links = s.get_rss(from_file=False, write=True)
breaking = s.breaking(write=True, mins=60)
news = s.get_news(write=True, mins=24*60)
df = pd.DataFrame(news, columns=s.news_schema)

CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 3.1 µs


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/sreetam/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
%%time
import numpy as np
from collections import Counter

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 5.72 µs


In [5]:
df['key_words'] = df['title'].apply(lambda x:s.pre_process(x))

In [6]:
all_dfs = []
for country in s.countries:
        for category in s.interests:
                n_df = df.loc[df['country']==country]
                unique, counts = np.unique(np.concatenate(n_df.key_words.values), return_counts=True)
                total_word_count = dict(sorted(dict(zip(unique, counts)).items(), 
                        key=lambda item: item[1], reverse=True))
                margin = max(total_word_count.values())*0.99
                trending_dict = dict((k, v) for k, v in total_word_count.items() if v >= margin)
                trending_words = list(trending_dict.keys())
                n_df = n_df.loc[[set(n_df.key_words[i])-set(trending_words)!=set(n_df.key_words[i]) for i in n_df.index]]
                all_dfs.append(n_df)

In [7]:
k_df = pd.concat(all_dfs)

In [10]:
k_df.title.values

array(["White House condemns North Korea's latest long-range missile launch",
       "House Republicans could pose threat to President Biden's agenda",
       'New generation of House Democrats vying for leadership spots',
       "Lauren Boebert's opponent concedes in tight Colorado House race",
       'Nancy Pelosi stands down as leader of US House Democrats',
       "House Republicans say 'top priority' is to probe Biden family",
       "3 House Republicans say they won't support McCarthy for speaker",
       'Democrats narrowly win control of PA State House',
       'Election denier Hakeem Jeffries could replace Pelosi as House Dem leader',
       'White House files Supreme Court emergency appeal to rescue $500B student debt handout',
       'Jordan seeks testimony from Garland, Wray, others in House Judiciary probes of DOJ, FBI',
       'Lauren Boebert’s Rival, Adam Frisch, Concedes Colorado House Race',
       'These Are the Democrats Who Could Replace Pelosi and Other House Leade